In [1]:
with open('a_example.txt', 'r') as f:
    content = f.read()
    
print(content)

6 2 7
1 2 3 6 5 4
5 2 2
0 1 2 3 4
4 3 1
0 2 3 5



In [181]:
def parseContent(fName):
    with open(fName, 'r') as f:
        content = f.read()
    lines = content.split('\n')
    info = {}
    info['nBooks'] = int(lines[0].split(' ')[0])
    info['nLibraries'] = int(lines[0].split(' ')[1])
    info['nDays'] = int(lines[0].split(' ')[2])
    
    bookScores = {}
    bookScoresLine = lines[1].split(' ')
    for i in range(len(bookScoresLine)):
        bookScores[i] = {'id': i, 'score': int(bookScoresLine[i])}
        
    libraryInfo = {}
    for i in range(info['nLibraries']):
        libraryInfoLine = [int(i) for i in lines[2*i + 2].split(' ')]
        libraryBooksInfoLine = [int(i) for i in lines[2*i + 3].split(' ')]
        currentLibraryInfo = {}
        currentLibraryInfo['nBooks'] = libraryInfoLine[0]
        currentLibraryInfo['signupTime'] = libraryInfoLine[1]
        currentLibraryInfo['shipQty'] = libraryInfoLine[2]
        currentLibraryInfo['books'] = {i for i in libraryBooksInfoLine}
        libraryInfo[i] = currentLibraryInfo
        
    return bookScores, info, libraryInfo

In [182]:
parseContent('a_example.txt')

({0: {'id': 0, 'score': 1},
  1: {'id': 1, 'score': 2},
  2: {'id': 2, 'score': 3},
  3: {'id': 3, 'score': 6},
  4: {'id': 4, 'score': 5},
  5: {'id': 5, 'score': 4}},
 {'nBooks': 6, 'nLibraries': 2, 'nDays': 7},
 {0: {'nBooks': 5, 'signupTime': 2, 'shipQty': 2, 'books': {0, 1, 2, 3, 4}},
  1: {'nBooks': 4, 'signupTime': 3, 'shipQty': 1, 'books': {0, 2, 3, 5}}})

In [183]:
def getOptimalOrder(lib, bookScores):
    def takeScore(el):
        return bookScores[el]['score']
    bookOrder = sorted(lib['books'], key=takeScore, reverse=True)
    return bookOrder

In [184]:
from math import ceil
def getTotalTimeToComplete(lib):
    return lib['signupTime'] + ceil(lib['nBooks']/lib['shipQty'])

In [191]:
def getLibMaxScoreAndOptimalOrder(lib, bookScores, daysToComplete):
    daysLeft = daysToComplete - lib['signupTime']
    
    if (daysLeft < 1):
        return 0, []
    bookOrder = getOptimalOrder(lib, bookScores)[0:daysLeft*lib['shipQty']]
    cumulativeScore = 0
    for i in range(daysLeft):
        booksForTheDay = bookOrder[lib['shipQty']*i: lib['shipQty']*i + lib['shipQty']]
        scoreForTheDay = sum([bookScores[i]['score'] for i in booksForTheDay])
        cumulativeScore += scoreForTheDay
    return cumulativeScore, bookOrder

In [192]:
getLibMaxScoreAndOptimalOrder({'nBooks': 4, 'signupTime': 3, 'shipQty': 1, 'books': [0, 2, 3, 5]}, {0: {'id': 0, 'score': 1},
  1: {'id': 1, 'score': 2},
  2: {'id': 2, 'score': 3},
  3: {'id': 3, 'score': 6},
  4: {'id': 4, 'score': 5},
  5: {'id': 5, 'score': 4}}, 7)

(14, [3, 5, 2, 0])

In [193]:
def getAverageBookScore(libInfo, bookScores):
    timeRequired = getTotalTimeToComplete(libInfo)
    totalScore = sum([int(bookScores[bookId]['score']) for bookId in libInfo['books']])
    return totalScore / timeRequired

In [194]:
getAverageBookScore({'nBooks': 4, 'signupTime': 3, 'shipQty': 1, 'books': [0, 2, 3, 5]}, 
{0: {'id': 0, 'score': 1},
  1: {'id': 1, 'score': 2},
  2: {'id': 2, 'score': 3},
  3: {'id': 3, 'score': 6},
  4: {'id': 4, 'score': 5},
  5: {'id': 5, 'score': 4}})

2.0

In [227]:
def generateResult(fName, outName):
    bookScores, info, libraryInfo = parseContent(fName)
    print('Parsed content')
    
    maxDays = info['nDays']
    libMaxScores = {}
    libIds = {libId for (libId, libInfo) in libraryInfo.items()}
    print('Got all library IDS', len(libIds))
    
    def takeAverage(libId):
        return getAverageBookScore(libraryInfo[libId], bookScores)
    sortedLibIds = sorted(libIds, key=takeAverage, reverse=True)

#     def takeSignupTime(libId):
#         return libraryInfo[libId]['signupTime']
#     sortedLibs = sorted(libIds, key=takeSignupTime)
    
    counter = 0
    daysPassed = 0
    finalLibIds = []
    for libId in sortedLibIds:
        currentLibInfo = libraryInfo[libId]
        currentLibMaxScore, currentLibOrder =  getLibMaxScoreAndOptimalOrder(
            currentLibInfo, bookScores, maxDays - daysPassed)
        if currentLibMaxScore > 0:
            libMaxScores[libId] = {
                'score': currentLibMaxScore,
                'order': currentLibOrder
            }
            finalLibIds.append(libId)
#             for libId in sortedLibIds:
#                 libraryInfo[libId]['books'] = {e for e in libraryInfo[libId]['books'] if e not in {a for a in currentLibOrder}}
        print('{}/{}'.format(counter, len(libIds)))
        counter += 1 
        daysPassed += currentLibInfo['signupTime']
        print('Day: {}'.format(daysPassed))
        if daysPassed > maxDays:
            break
    print('Got scores and order for every library')
    
    def takeScore(libId):
        return libMaxScores[libId]['score']
    finalLibIds = sorted(finalLibIds, key=takeScore, reverse=True)
        
    nLibs = len(finalLibIds)
    with open(outName, 'w+') as f:
        f.write(str(nLibs) + '\n')
        for libId in finalLibIds:
            f.write('{} {}\n'.format(libId, len(libMaxScores[libId]['order'])))
            f.write(' '.join([str(i) for i in libMaxScores[libId]['order']]) + '\n')
            
    print('Done')

In [228]:
generateResult('a_example.txt', 'a_example_result.txt')

Parsed content
Got all library IDS 2
0/2
Day: 2
1/2
Day: 5
Got scores and order for every library
Done


In [229]:
generateResult('b_read_on.txt', 'b_read_on_result.txt')

Parsed content
Got all library IDS 100
0/100
Day: 1
1/100
Day: 2
2/100
Day: 4
3/100
Day: 6
4/100
Day: 8
5/100
Day: 10
6/100
Day: 12
7/100
Day: 15
8/100
Day: 18
9/100
Day: 21
10/100
Day: 24
11/100
Day: 28
12/100
Day: 32
13/100
Day: 36
14/100
Day: 40
15/100
Day: 44
16/100
Day: 49
17/100
Day: 54
18/100
Day: 59
19/100
Day: 64
20/100
Day: 70
21/100
Day: 76
22/100
Day: 82
23/100
Day: 88
24/100
Day: 95
25/100
Day: 102
26/100
Day: 109
27/100
Day: 116
28/100
Day: 124
29/100
Day: 132
30/100
Day: 140
31/100
Day: 148
32/100
Day: 156
33/100
Day: 164
34/100
Day: 172
35/100
Day: 180
36/100
Day: 188
37/100
Day: 197
38/100
Day: 207
39/100
Day: 217
40/100
Day: 227
41/100
Day: 237
42/100
Day: 247
43/100
Day: 258
44/100
Day: 269
45/100
Day: 281
46/100
Day: 293
47/100
Day: 305
48/100
Day: 317
49/100
Day: 329
50/100
Day: 342
51/100
Day: 355
52/100
Day: 369
53/100
Day: 383
54/100
Day: 397
55/100
Day: 411
56/100
Day: 426
57/100
Day: 441
58/100
Day: 456
59/100
Day: 471
60/100
Day: 486
61/100
Day: 501
62/100
Da

In [219]:
generateResult('c_incunabula.txt', 'c_incunabula_result.txt')

Parsed content
Got all library IDS 10000
0/10000
Day: 10
1/10000
Day: 21
2/10000
Day: 31
3/10000
Day: 42
4/10000
Day: 52
5/10000
Day: 62
6/10000
Day: 75
7/10000
Day: 85
8/10000
Day: 95
9/10000
Day: 108
10/10000
Day: 122
11/10000
Day: 136
12/10000
Day: 151
13/10000
Day: 164
14/10000
Day: 176
15/10000
Day: 190
16/10000
Day: 201
17/10000
Day: 211
18/10000
Day: 226
19/10000
Day: 237
20/10000
Day: 255
21/10000
Day: 271
22/10000
Day: 283
23/10000
Day: 293
24/10000
Day: 306
25/10000
Day: 316
26/10000
Day: 332
27/10000
Day: 347
28/10000
Day: 359
29/10000
Day: 374
30/10000
Day: 387
31/10000
Day: 400
32/10000
Day: 416
33/10000
Day: 427
34/10000
Day: 447
35/10000
Day: 459
36/10000
Day: 469
37/10000
Day: 480
38/10000
Day: 498
39/10000
Day: 515
40/10000
Day: 526
41/10000
Day: 541
42/10000
Day: 553
43/10000
Day: 567
44/10000
Day: 585
45/10000
Day: 598
46/10000
Day: 609
47/10000
Day: 630
48/10000
Day: 645
49/10000
Day: 664
50/10000
Day: 675
51/10000
Day: 688
52/10000
Day: 707
53/10000
Day: 719
54/100

415/10000
Day: 13283
416/10000
Day: 13327
417/10000
Day: 13389
418/10000
Day: 13431
419/10000
Day: 13488
420/10000
Day: 13539
421/10000
Day: 13604
422/10000
Day: 13642
423/10000
Day: 13701
424/10000
Day: 13746
425/10000
Day: 13801
426/10000
Day: 13834
427/10000
Day: 13888
428/10000
Day: 13949
429/10000
Day: 14004
430/10000
Day: 14056
431/10000
Day: 14141
432/10000
Day: 14209
433/10000
Day: 14253
434/10000
Day: 14328
435/10000
Day: 14386
436/10000
Day: 14427
437/10000
Day: 14472
438/10000
Day: 14556
439/10000
Day: 14620
440/10000
Day: 14661
441/10000
Day: 14698
442/10000
Day: 14777
443/10000
Day: 14824
444/10000
Day: 14898
445/10000
Day: 14954
446/10000
Day: 15016
447/10000
Day: 15068
448/10000
Day: 15109
449/10000
Day: 15171
450/10000
Day: 15203
451/10000
Day: 15254
452/10000
Day: 15314
453/10000
Day: 15389
454/10000
Day: 15424
455/10000
Day: 15458
456/10000
Day: 15519
457/10000
Day: 15567
458/10000
Day: 15626
459/10000
Day: 15666
460/10000
Day: 15710
461/10000
Day: 15780
462/10000
Day

806/10000
Day: 41063
807/10000
Day: 41154
808/10000
Day: 41215
809/10000
Day: 41297
810/10000
Day: 41390
811/10000
Day: 41494
812/10000
Day: 41573
813/10000
Day: 41694
814/10000
Day: 41782
815/10000
Day: 41849
816/10000
Day: 41916
817/10000
Day: 41995
818/10000
Day: 42097
819/10000
Day: 42150
820/10000
Day: 42248
821/10000
Day: 42387
822/10000
Day: 42498
823/10000
Day: 42567
824/10000
Day: 42666
825/10000
Day: 42732
826/10000
Day: 42796
827/10000
Day: 42885
828/10000
Day: 42997
829/10000
Day: 43083
830/10000
Day: 43141
831/10000
Day: 43213
832/10000
Day: 43285
833/10000
Day: 43399
834/10000
Day: 43543
835/10000
Day: 43635
836/10000
Day: 43771
837/10000
Day: 43865
838/10000
Day: 43927
839/10000
Day: 44062
840/10000
Day: 44113
841/10000
Day: 44202
842/10000
Day: 44275
843/10000
Day: 44422
844/10000
Day: 44531
845/10000
Day: 44614
846/10000
Day: 44746
847/10000
Day: 44868
848/10000
Day: 44976
849/10000
Day: 45018
850/10000
Day: 45133
851/10000
Day: 45193
852/10000
Day: 45309
853/10000
Day

1204/10000
Day: 86727
1205/10000
Day: 86830
1206/10000
Day: 86973
1207/10000
Day: 87119
1208/10000
Day: 87230
1209/10000
Day: 87360
1210/10000
Day: 87535
1211/10000
Day: 87688
1212/10000
Day: 87858
1213/10000
Day: 87938
1214/10000
Day: 88087
1215/10000
Day: 88256
1216/10000
Day: 88407
1217/10000
Day: 88555
1218/10000
Day: 88690
1219/10000
Day: 88837
1220/10000
Day: 88989
1221/10000
Day: 89123
1222/10000
Day: 89244
1223/10000
Day: 89450
1224/10000
Day: 89555
1225/10000
Day: 89640
1226/10000
Day: 89834
1227/10000
Day: 90027
1228/10000
Day: 90161
1229/10000
Day: 90363
1230/10000
Day: 90530
1231/10000
Day: 90660
1232/10000
Day: 90793
1233/10000
Day: 90961
1234/10000
Day: 91095
1235/10000
Day: 91248
1236/10000
Day: 91406
1237/10000
Day: 91560
1238/10000
Day: 91649
1239/10000
Day: 91742
1240/10000
Day: 91917
1241/10000
Day: 92017
1242/10000
Day: 92166
1243/10000
Day: 92377
1244/10000
Day: 92589
1245/10000
Day: 92733
1246/10000
Day: 92875
1247/10000
Day: 93105
1248/10000
Day: 93205
1249/10000

In [220]:
generateResult('d_tough_choices.txt', 'd_tough_choices_result.txt')

Parsed content
Got all library IDS 30000
0/30000
Day: 2
1/30000
Day: 4
2/30000
Day: 6
3/30000
Day: 8
4/30000
Day: 10
5/30000
Day: 12
6/30000
Day: 14
7/30000
Day: 16
8/30000
Day: 18
9/30000
Day: 20
10/30000
Day: 22
11/30000
Day: 24
12/30000
Day: 26
13/30000
Day: 28
14/30000
Day: 30
15/30000
Day: 32
16/30000
Day: 34
17/30000
Day: 36
18/30000
Day: 38
19/30000
Day: 40
20/30000
Day: 42
21/30000
Day: 44
22/30000
Day: 46
23/30000
Day: 48
24/30000
Day: 50
25/30000
Day: 52
26/30000
Day: 54
27/30000
Day: 56
28/30000
Day: 58
29/30000
Day: 60
30/30000
Day: 62
31/30000
Day: 64
32/30000
Day: 66
33/30000
Day: 68
34/30000
Day: 70
35/30000
Day: 72
36/30000
Day: 74
37/30000
Day: 76
38/30000
Day: 78
39/30000
Day: 80
40/30000
Day: 82
41/30000
Day: 84
42/30000
Day: 86
43/30000
Day: 88
44/30000
Day: 90
45/30000
Day: 92
46/30000
Day: 94
47/30000
Day: 96
48/30000
Day: 98
49/30000
Day: 100
50/30000
Day: 102
51/30000
Day: 104
52/30000
Day: 106
53/30000
Day: 108
54/30000
Day: 110
55/30000
Day: 112
56/30000
Day: 

443/30000
Day: 888
444/30000
Day: 890
445/30000
Day: 892
446/30000
Day: 894
447/30000
Day: 896
448/30000
Day: 898
449/30000
Day: 900
450/30000
Day: 902
451/30000
Day: 904
452/30000
Day: 906
453/30000
Day: 908
454/30000
Day: 910
455/30000
Day: 912
456/30000
Day: 914
457/30000
Day: 916
458/30000
Day: 918
459/30000
Day: 920
460/30000
Day: 922
461/30000
Day: 924
462/30000
Day: 926
463/30000
Day: 928
464/30000
Day: 930
465/30000
Day: 932
466/30000
Day: 934
467/30000
Day: 936
468/30000
Day: 938
469/30000
Day: 940
470/30000
Day: 942
471/30000
Day: 944
472/30000
Day: 946
473/30000
Day: 948
474/30000
Day: 950
475/30000
Day: 952
476/30000
Day: 954
477/30000
Day: 956
478/30000
Day: 958
479/30000
Day: 960
480/30000
Day: 962
481/30000
Day: 964
482/30000
Day: 966
483/30000
Day: 968
484/30000
Day: 970
485/30000
Day: 972
486/30000
Day: 974
487/30000
Day: 976
488/30000
Day: 978
489/30000
Day: 980
490/30000
Day: 982
491/30000
Day: 984
492/30000
Day: 986
493/30000
Day: 988
494/30000
Day: 990
495/30000
Da

858/30000
Day: 1718
859/30000
Day: 1720
860/30000
Day: 1722
861/30000
Day: 1724
862/30000
Day: 1726
863/30000
Day: 1728
864/30000
Day: 1730
865/30000
Day: 1732
866/30000
Day: 1734
867/30000
Day: 1736
868/30000
Day: 1738
869/30000
Day: 1740
870/30000
Day: 1742
871/30000
Day: 1744
872/30000
Day: 1746
873/30000
Day: 1748
874/30000
Day: 1750
875/30000
Day: 1752
876/30000
Day: 1754
877/30000
Day: 1756
878/30000
Day: 1758
879/30000
Day: 1760
880/30000
Day: 1762
881/30000
Day: 1764
882/30000
Day: 1766
883/30000
Day: 1768
884/30000
Day: 1770
885/30000
Day: 1772
886/30000
Day: 1774
887/30000
Day: 1776
888/30000
Day: 1778
889/30000
Day: 1780
890/30000
Day: 1782
891/30000
Day: 1784
892/30000
Day: 1786
893/30000
Day: 1788
894/30000
Day: 1790
895/30000
Day: 1792
896/30000
Day: 1794
897/30000
Day: 1796
898/30000
Day: 1798
899/30000
Day: 1800
900/30000
Day: 1802
901/30000
Day: 1804
902/30000
Day: 1806
903/30000
Day: 1808
904/30000
Day: 1810
905/30000
Day: 1812
906/30000
Day: 1814
907/30000
Day: 1816


1259/30000
Day: 2520
1260/30000
Day: 2522
1261/30000
Day: 2524
1262/30000
Day: 2526
1263/30000
Day: 2528
1264/30000
Day: 2530
1265/30000
Day: 2532
1266/30000
Day: 2534
1267/30000
Day: 2536
1268/30000
Day: 2538
1269/30000
Day: 2540
1270/30000
Day: 2542
1271/30000
Day: 2544
1272/30000
Day: 2546
1273/30000
Day: 2548
1274/30000
Day: 2550
1275/30000
Day: 2552
1276/30000
Day: 2554
1277/30000
Day: 2556
1278/30000
Day: 2558
1279/30000
Day: 2560
1280/30000
Day: 2562
1281/30000
Day: 2564
1282/30000
Day: 2566
1283/30000
Day: 2568
1284/30000
Day: 2570
1285/30000
Day: 2572
1286/30000
Day: 2574
1287/30000
Day: 2576
1288/30000
Day: 2578
1289/30000
Day: 2580
1290/30000
Day: 2582
1291/30000
Day: 2584
1292/30000
Day: 2586
1293/30000
Day: 2588
1294/30000
Day: 2590
1295/30000
Day: 2592
1296/30000
Day: 2594
1297/30000
Day: 2596
1298/30000
Day: 2598
1299/30000
Day: 2600
1300/30000
Day: 2602
1301/30000
Day: 2604
1302/30000
Day: 2606
1303/30000
Day: 2608
1304/30000
Day: 2610
1305/30000
Day: 2612
1306/30000
Da

1652/30000
Day: 3306
1653/30000
Day: 3308
1654/30000
Day: 3310
1655/30000
Day: 3312
1656/30000
Day: 3314
1657/30000
Day: 3316
1658/30000
Day: 3318
1659/30000
Day: 3320
1660/30000
Day: 3322
1661/30000
Day: 3324
1662/30000
Day: 3326
1663/30000
Day: 3328
1664/30000
Day: 3330
1665/30000
Day: 3332
1666/30000
Day: 3334
1667/30000
Day: 3336
1668/30000
Day: 3338
1669/30000
Day: 3340
1670/30000
Day: 3342
1671/30000
Day: 3344
1672/30000
Day: 3346
1673/30000
Day: 3348
1674/30000
Day: 3350
1675/30000
Day: 3352
1676/30000
Day: 3354
1677/30000
Day: 3356
1678/30000
Day: 3358
1679/30000
Day: 3360
1680/30000
Day: 3362
1681/30000
Day: 3364
1682/30000
Day: 3366
1683/30000
Day: 3368
1684/30000
Day: 3370
1685/30000
Day: 3372
1686/30000
Day: 3374
1687/30000
Day: 3376
1688/30000
Day: 3378
1689/30000
Day: 3380
1690/30000
Day: 3382
1691/30000
Day: 3384
1692/30000
Day: 3386
1693/30000
Day: 3388
1694/30000
Day: 3390
1695/30000
Day: 3392
1696/30000
Day: 3394
1697/30000
Day: 3396
1698/30000
Day: 3398
1699/30000
Da

...

12960/30000
Day: 25922
12961/30000
Day: 25924
12962/30000
Day: 25926
12963/30000
Day: 25928
12964/30000
Day: 25930
12965/30000
Day: 25932
12966/30000
Day: 25934
12967/30000
Day: 25936
12968/30000
Day: 25938
12969/30000
Day: 25940
12970/30000
Day: 25942
12971/30000
Day: 25944
12972/30000
Day: 25946
12973/30000
Day: 25948
12974/30000
Day: 25950
12975/30000
Day: 25952
12976/30000
Day: 25954
12977/30000
Day: 25956
12978/30000
Day: 25958
12979/30000
Day: 25960
12980/30000
Day: 25962
12981/30000
Day: 25964
12982/30000
Day: 25966
12983/30000
Day: 25968
12984/30000
Day: 25970
12985/30000
Day: 25972
12986/30000
Day: 25974
12987/30000
Day: 25976
12988/30000
Day: 25978
12989/30000
Day: 25980
12990/30000
Day: 25982
12991/30000
Day: 25984
12992/30000
Day: 25986
12993/30000
Day: 25988
12994/30000
Day: 25990
12995/30000
Day: 25992
12996/30000
Day: 25994
12997/30000
Day: 25996
12998/30000
Day: 25998
12999/30000
Day: 26000
13000/30000
Day: 26002
13001/30000
Day: 26004
13002/30000
Day: 26006
13003/30000

13428/30000
Day: 26858
13429/30000
Day: 26860
13430/30000
Day: 26862
13431/30000
Day: 26864
13432/30000
Day: 26866
13433/30000
Day: 26868
13434/30000
Day: 26870
13435/30000
Day: 26872
13436/30000
Day: 26874
13437/30000
Day: 26876
13438/30000
Day: 26878
13439/30000
Day: 26880
13440/30000
Day: 26882
13441/30000
Day: 26884
13442/30000
Day: 26886
13443/30000
Day: 26888
13444/30000
Day: 26890
13445/30000
Day: 26892
13446/30000
Day: 26894
13447/30000
Day: 26896
13448/30000
Day: 26898
13449/30000
Day: 26900
13450/30000
Day: 26902
13451/30000
Day: 26904
13452/30000
Day: 26906
13453/30000
Day: 26908
13454/30000
Day: 26910
13455/30000
Day: 26912
13456/30000
Day: 26914
13457/30000
Day: 26916
13458/30000
Day: 26918
13459/30000
Day: 26920
13460/30000
Day: 26922
13461/30000
Day: 26924
13462/30000
Day: 26926
13463/30000
Day: 26928
13464/30000
Day: 26930
13465/30000
Day: 26932
13466/30000
Day: 26934
13467/30000
Day: 26936
13468/30000
Day: 26938
13469/30000
Day: 26940
13470/30000
Day: 26942
13471/30000

13804/30000
Day: 27610
13805/30000
Day: 27612
13806/30000
Day: 27614
13807/30000
Day: 27616
13808/30000
Day: 27618
13809/30000
Day: 27620
13810/30000
Day: 27622
13811/30000
Day: 27624
13812/30000
Day: 27626
13813/30000
Day: 27628
13814/30000
Day: 27630
13815/30000
Day: 27632
13816/30000
Day: 27634
13817/30000
Day: 27636
13818/30000
Day: 27638
13819/30000
Day: 27640
13820/30000
Day: 27642
13821/30000
Day: 27644
13822/30000
Day: 27646
13823/30000
Day: 27648
13824/30000
Day: 27650
13825/30000
Day: 27652
13826/30000
Day: 27654
13827/30000
Day: 27656
13828/30000
Day: 27658
13829/30000
Day: 27660
13830/30000
Day: 27662
13831/30000
Day: 27664
13832/30000
Day: 27666
13833/30000
Day: 27668
13834/30000
Day: 27670
13835/30000
Day: 27672
13836/30000
Day: 27674
13837/30000
Day: 27676
13838/30000
Day: 27678
13839/30000
Day: 27680
13840/30000
Day: 27682
13841/30000
Day: 27684
13842/30000
Day: 27686
13843/30000
Day: 27688
13844/30000
Day: 27690
13845/30000
Day: 27692
13846/30000
Day: 27694
13847/30000

14245/30000
Day: 28492
14246/30000
Day: 28494
14247/30000
Day: 28496
14248/30000
Day: 28498
14249/30000
Day: 28500
14250/30000
Day: 28502
14251/30000
Day: 28504
14252/30000
Day: 28506
14253/30000
Day: 28508
14254/30000
Day: 28510
14255/30000
Day: 28512
14256/30000
Day: 28514
14257/30000
Day: 28516
14258/30000
Day: 28518
14259/30000
Day: 28520
14260/30000
Day: 28522
14261/30000
Day: 28524
14262/30000
Day: 28526
14263/30000
Day: 28528
14264/30000
Day: 28530
14265/30000
Day: 28532
14266/30000
Day: 28534
14267/30000
Day: 28536
14268/30000
Day: 28538
14269/30000
Day: 28540
14270/30000
Day: 28542
14271/30000
Day: 28544
14272/30000
Day: 28546
14273/30000
Day: 28548
14274/30000
Day: 28550
14275/30000
Day: 28552
14276/30000
Day: 28554
14277/30000
Day: 28556
14278/30000
Day: 28558
14279/30000
Day: 28560
14280/30000
Day: 28562
14281/30000
Day: 28564
14282/30000
Day: 28566
14283/30000
Day: 28568
14284/30000
Day: 28570
14285/30000
Day: 28572
14286/30000
Day: 28574
14287/30000
Day: 28576
14288/30000

14938/30000
Day: 29878
14939/30000
Day: 29880
14940/30000
Day: 29882
14941/30000
Day: 29884
14942/30000
Day: 29886
14943/30000
Day: 29888
14944/30000
Day: 29890
14945/30000
Day: 29892
14946/30000
Day: 29894
14947/30000
Day: 29896
14948/30000
Day: 29898
14949/30000
Day: 29900
14950/30000
Day: 29902
14951/30000
Day: 29904
14952/30000
Day: 29906
14953/30000
Day: 29908
14954/30000
Day: 29910
14955/30000
Day: 29912
14956/30000
Day: 29914
14957/30000
Day: 29916
14958/30000
Day: 29918
14959/30000
Day: 29920
14960/30000
Day: 29922
14961/30000
Day: 29924
14962/30000
Day: 29926
14963/30000
Day: 29928
14964/30000
Day: 29930
14965/30000
Day: 29932
14966/30000
Day: 29934
14967/30000
Day: 29936
14968/30000
Day: 29938
14969/30000
Day: 29940
14970/30000
Day: 29942
14971/30000
Day: 29944
14972/30000
Day: 29946
14973/30000
Day: 29948
14974/30000
Day: 29950
14975/30000
Day: 29952
14976/30000
Day: 29954
14977/30000
Day: 29956
14978/30000
Day: 29958
14979/30000
Day: 29960
14980/30000
Day: 29962
14981/30000

In [230]:
generateResult('e_so_many_books.txt', 'e_so_many_books_result.txt')

Parsed content
Got all library IDS 1000
0/1000
Day: 1
1/1000
Day: 2
2/1000
Day: 3
3/1000
Day: 6
4/1000
Day: 10
5/1000
Day: 13
6/1000
Day: 16
7/1000
Day: 19
8/1000
Day: 20
9/1000
Day: 21
10/1000
Day: 22
11/1000
Day: 23
12/1000
Day: 30
13/1000
Day: 36
14/1000
Day: 41
15/1000
Day: 42
16/1000
Day: 43
17/1000
Day: 44
18/1000
Day: 46
19/1000
Day: 47
20/1000
Day: 50
21/1000
Day: 53
22/1000
Day: 58
23/1000
Day: 61
24/1000
Day: 65
25/1000
Day: 67
26/1000
Day: 71
27/1000
Day: 73
28/1000
Day: 82
29/1000
Day: 83
30/1000
Day: 84
31/1000
Day: 86
32/1000
Day: 88
33/1000
Day: 91
34/1000
Day: 93
35/1000
Day: 100
36/1000
Day: 106
37/1000
Day: 110
38/1000
Day: 114
39/1000
Day: 121
40/1000
Day: 123
41/1000
Day: 133
42/1000
Day: 135
43/1000
Day: 142
44/1000
Day: 144
45/1000
Day: 146
46/1000
Day: 147
47/1000
Day: 153
48/1000
Day: 158
49/1000
Day: 159
50/1000
Day: 166
51/1000
Day: 168
52/1000
Day: 169
53/1000
Day: 171
54/1000
Day: 174
55/1000
Day: 176
56/1000
Day: 179
57/1000
Day: 183
58/1000
Day: 186
59/100

In [222]:
generateResult('f_libraries_of_the_world.txt', 'f_libraries_of_the_world_result.txt')

Parsed content
Got all library IDS 1000
0/1000
Day: 40
1/1000
Day: 70
2/1000
Day: 114
3/1000
Day: 159
4/1000
Day: 210
5/1000
Day: 258
6/1000
Day: 299
7/1000
Day: 348
8/1000
Day: 397
9/1000
Day: 447
10/1000
Day: 519
11/1000
Day: 569
12/1000
Day: 611
13/1000
Day: 643
14/1000
Day: 708
Got scores and order for every library
Done


In [108]:
bookScores, info, libraryInfo = parseContent('c_incunabula.txt')

In [103]:
import pandas as pd

In [109]:
df = pd.DataFrame.from_dict(libraryInfo)

In [110]:
df

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
nBooks,13,19,18,14,12,16,15,20,12,19,...,20,17,17,16,14,18,16,12,19,14
signupTime,344,966,1000,276,869,105,807,737,278,400,...,824,902,75,336,261,648,736,668,955,664
shipQty,33474,53538,27830,8039,717,17003,65258,46446,67429,1890,...,68688,21280,72514,26072,27232,69743,17734,90986,4483,47542
books,"[45608, 94192, 70546, 4346, 45948, 94356, 6990...","[30294, 64718, 22277, 51943, 75323, 26185, 431...","[99122, 81548, 95109, 7100, 35387, 2833, 88815...","[49233, 54205, 19769, 93556, 4982, 43193, 2341...","[88055, 84490, 39112, 16808, 75782, 24927, 141...","[31929, 32713, 47081, 83777, 14459, 30091, 699...","[47182, 46, 2535, 32000, 78614, 27899, 8469, 7...","[37050, 47086, 22656, 27984, 79184, 70688, 370...","[71854, 47723, 53591, 82683, 44699, 6488, 9995...","[90178, 64402, 32264, 35949, 95375, 91115, 736...",...,"[78087, 18639, 97012, 74101, 90932, 68991, 959...","[19210, 30752, 51502, 9968, 72860, 33383, 9886...","[21906, 65970, 97315, 18310, 19245, 12834, 638...","[79752, 44185, 63913, 13809, 72203, 27075, 551...","[75783, 20708, 66894, 50457, 81303, 33344, 517...","[70893, 46238, 1518, 37631, 49403, 89542, 8636...","[83827, 6242, 85586, 31883, 43344, 6423, 67187...","[10870, 28529, 44061, 69237, 47148, 53944, 855...","[64416, 73064, 53242, 865, 93052, 85931, 17066...","[47547, 47804, 27423, 52506, 25732, 19634, 693..."
